<a href="https://colab.research.google.com/github/LukDayran/hello-remote/blob/master/PythonAlina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Викторина 
## Как программа работает?
Программа считывает вопросы, возможные ответы и баллы за них из файла, преобразуя это все в словарь (внутри которого еще один словарь)

Дальше, программа задает пользователю вопросы, при получении ответа он его сохраняет и проверяет на правильность. Если ответ неправильный, то задает еще N количетво раз, снижая возможные баллы за ответ. Колличество попыток можно легко изменить. В ситуации отсутствия правильного ответа, пользователю засчитывается 0 баллов и выдается None в качестве как бы ответа.

Затем, мы записываем уже в новый файл (который сам создастся, если отсутсвует) все результаты: вопросы, ответы, баллы за них, индекс успешности и как бы оценка, хорошо или плохо.

##Особенности структуры кода
Изначальный словарь и записывание ответов в файл происходит с помощью вызоыва функции. Основное тело программы занимает именно цикл задачи вопросов и записи ответов и баллов.

##Что ожидать пользователю
Вначале программа рассказывает, что его ждет. Озвучивает каждый вопрос и возможные ответы и в случае неверного ответа просит еще раз попробовать. Ответы можно вводить как строчными, так и прописными. 


In [ ]:
import os


# В данной функции мы считывает из файлы вопросы, ответы и баллы и создаем из них словарь словарей
def fill_quiz_dict(quiz_file):
    quiz_dict = {}
    with open(quiz_file, "r") as f:
        for line in f:
            parts = line.strip().split(",") #делим каждую страку на части по запятой
            question = parts[0] #Вопрос всегда первый
            answers_dict = {}
            for i in range(1, len(parts), 2): #тут мы создаем словарь с вопросами и их баллами
                answer = parts[i].lower() 
                points = int(parts[i + 1])
                answers_dict[answer] = points
            quiz_dict[question] = answers_dict #...и тут добавляет его в основной большой словарь
    return quiz_dict
#Тут мы записываем все имеющиеся у нас данные в отдельный файл, при его отсутсвии создаем его (для этого нам import os и нужен)
#Вопрос, финальный ответ, баллы, все ответы, оценку работе
def print_results(user_dict):
    with open("quiz_results.txt", "w") as f:
        f.write("Ваши ответы:\n")
        total_points = 0
        for question, data_dict in user_dict.items():
            f.write(question + "\n")
            f.write("Ваш ответ: " + data_dict["answer"] + "\n")
            f.write("Баллы: " + str(data_dict["points"]) + "\n")
            f.write("Все ответы: " + ", ".join(data_dict["user_answers"]) + "\n")
            f.write("\n")
            total_points += data_dict["points"]
        success_rate = round(total_points / len(user_dict), 1) #складывая все баллы и зная количество вопросов получает индекс усспешности ответов
        f.write("Ваш индекс успешных ответов - " + str(success_rate) + "\n")
        if success_rate == 1:
            f.write("Идеально!\n")
        elif success_rate >= 0.5 and success_rate < 1:
            f.write("Хорошо\n")
        else:
            f.write("Плохо\n")
#указываем название изначального файла, используем ранеее описанную функцию и создаем новый словарь уже с ответами пользователя
quiz_file = "quiz_data.txt" #название файда со всеми изначальными данными в формате "вопрос,ответ,балл"
quiz_dict = fill_quiz_dict(quiz_file)
user_dict = {}


print('Вас ожидает серия вопросов. На каждый вопрос у вас есть ', 2,' попыток. С каждым неправильным ответов ваш балл уменьшается, что вляет на финальный результат. Ввод ответа регистронезависимый')
for question, answers_dict in quiz_dict.items():
    print(question)
    for answer, points in answers_dict.items(): #печатаем вопросы
        print(answer)
    num_chances = 2
    decrease_index = round(1/num_chances, 1) #так мы можем уменьшать баллы за неправильные ответы
    #резонно уменьшать основываясь на колличестве попыток.      
    points_decrease = 0 
    user_answers = []
    while num_chances > 0:
        user_answer = input("Введите свой ответ: ").lower() 
        user_answers.append(user_answer)
        #добавляет ответ пользователя в список всех ответов; если ответ правильный, начисляем балл и запускаем новый этап цикла.
        if user_answer in answers_dict and answers_dict[user_answer] == 1:
            if question not in user_dict:
                user_dict[question] = {}
            user_dict[question]["answer"] = user_answer
            user_dict[question]["points"] = answers_dict[user_answer] - points_decrease
            break
        #Если же ответ неверный, то если есть попытки, даем еще шанс, уменьшая баллы. Если кончились, даем 0 баллов и в качестве ответа записывает None    
        else:
            num_chances -= 1
            points_decrease += decrease_index
            if num_chances > 0:
                print("Ответ неверный. У вас осталось ", num_chances, "попыток.")
            else:
                print("Ответ неверный.")
                if question not in user_dict:
                    user_dict[question] = {}
                user_dict[question]["answer"] = "None"
                user_dict[question]["points"] = 0
    user_dict[question]["user_answers"] = user_answers #добавляем получееные ответы в словарь
#записываем в файл результаты, вызывая функцию и указываем путь, где будет храниться этот файл
print_results(user_dict)
print("Результаты записаны в следующий файл: ", os.path.abspath("quiz_results.txt"))
print('Спасибо за ваше участие!')
